# Getting Started with TensorFlow

## Importing TensorFlow

In [1]:
import tensorflow as tf

The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number dimensions.

## Building the computational graph
A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output.One type of node is a constant. it takes no inputs and it outputs a value it stores internally.

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


## Running the computational graph

To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime.
The following code creates a Session object and then invokes its run method to run enough of the computational graph to evaluate node1 and node2. By running the computational graph in a session as follows:

In [3]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


Lets combine those two nodes with operations, we can add our two constant nodes and produce a new graph as follows:

In [4]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3)", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3) 7.0


A graphe can be parametrized to accept external inputs, known as placeholders. A placeholder is promise to provide a value late

In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + provides a shortcut for tf.add(a, b)

In [6]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


We can make the computational graph more complex by adding another operation:

In [7]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


To make the model trainable, we nned to be able to modify the graph to get new outpus with the same input, Varibales allow us to add trainable paramters to graph

In [9]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

Variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [10]:
init = tf.global_variables_initializer()
sess.run(init)

Since x is a placeholder, we can evaluate linear_model for several values of x simultaneously as follows:

In [11]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


# Evaluate the model
To evaluate the model on training data, we need a y placeholder to provide the desired values, and we need to write a loss function.
A loss function measures how far apart the current model is from the profided data.

In [13]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y) # we create a vector where each element is the corresponding examples error deltat
loss = tf.reduce_sum(squared_deltas) # we sum all squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


We can change W and b to the perfect values:

In [14]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


# Training the model
We guessed the "perfect" values of W and b, but the whole point of machine learning is to find the correct model parameters automatically. TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. The simplest optimizer is gradient descent. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable.


In [15]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [16]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [20]:
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
print("W: %s b: %s loss: %s" % (curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11
